In [2]:
import os
import pandas as pd
from pathlib import Path
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score
import optuna
import deslib.des.knora_u as des
import deslib.des.knora_e as knora_e
import deslib.des.des_p as desp
import deslib.static as static
import deslib.des.meta_des as meta
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path = Path(os.path.abspath(''))
data_path = path.resolve().parents[0]/'data'
with open(data_path/'pool_classifiers.pkl', 'rb') as model_file:
    pool = pickle.load(model_file)

In [6]:
train = pd.read_pickle(data_path/'train_OHE.pkl')
validation = pd.read_pickle(data_path/'validation_OHE.pkl')
train_x = train.drop(['label'], axis = 1)
train_y = train['label']
validation_x = validation.drop(['label'], axis = 1)
validation_y = validation['label']

In [7]:
test_OHE = pd.read_pickle(data_path/'test_final_OHE.pkl')
test_X = test_OHE.drop(['label'], axis = 1)
test_Y = test_OHE['label']

In [8]:
def probs(model,xtest):
    probs = model.predict_proba(xtest)
    if len(probs[0])>1:
        return [prob[1] for prob in probs]
    return probs

In [9]:
def perf_report(probs, testy):
    print(f"ROC_AUC is {roc_auc_score(y_true = testy, y_score = probs)}")
    print(f"PR_AUC is {average_precision_score(y_true = testy, y_score = probs)}")


In [26]:
pool_probs = probs(pool, test_X)

In [27]:
# performance of 50 bagged trees
roc_auc_score(y_true = test_Y, y_score = pool_probs)

0.8880216305468979

In [174]:
average_precision_score(y_true= test_Y, y_score= pool_probs)

0.3790472567667988

In [22]:
with open(data_path/'knu_homo_baggedtrees.pkl', 'rb') as model_file:
    ens = pickle.load(model_file)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator BaggingClassifier from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.1 when using version 1

In [28]:
ens_probs = probs(ens,test_X)

In [30]:
# performance of KNU using 50 bagged trees
roc_auc_score(y_true = test_Y, y_score = ens_probs)

0.8879578922910064

In [175]:
average_precision_score(y_true= test_Y, y_score= ens_probs)

0.3844581832481087

In [31]:
with open(data_path/'knu_hetero_rf_nn_NB.pkl', 'rb') as model_file:
    het_ens = pickle.load(model_file)

In [32]:
het_ens_probs = probs(het_ens, test_X)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [33]:
# performance of hetero ensemble using nn, rf and NB
roc_auc_score(y_true = test_Y, y_score = het_ens_probs)

0.8864593346432779

In [176]:
average_precision_score(y_true= test_Y, y_score= het_ens_probs)

0.4032371269903813

In [23]:
with open(data_path/'rf.pkl', 'rb') as model_file:
    rf = pickle.load(model_file)

In [24]:
with open(data_path/'nn.pkl', 'rb') as model_file:
    nn = pickle.load(model_file)

In [25]:
with open(data_path/'naive_bayes.pkl', 'rb') as model_file:
    NB = pickle.load(model_file)

In [42]:
classifiers = [rf, nn, NB, pool]

In [77]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,20)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = des.KNORAU(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state = 42)
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorau',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-10-30 14:45:50,442] A new study created in memory with name: heterogeneous_knorau


In [78]:
study.optimize(lambda trial: objective(trial, classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

[W 2023-10-30 14:45:53,003] Trial 0 failed with parameters: {'k': 19, 'ih_ind': False, 'dfp': True, 'ih_rate': 0.477447244685287} because of the following error: ValueError('Invalid value for parameter "voting". "voting" should be one of these options {selection, hybrid, weighting}').
Traceback (most recent call last):
  File "c:\Users\65829\anaconda3\envs\wp\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\65829\AppData\Local\Temp\ipykernel_23688\2213714495.py", line 1, in <lambda>
    study.optimize(lambda trial: objective(trial, classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)
  File "C:\Users\65829\AppData\Local\Temp\ipykernel_23688\1178912775.py", line 7, in objective
    knu.fit(X_train, y_train)
  File "c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\base.py", line 120, in fit
    self._validate_parameters()
  File "c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\

ValueError: Invalid value for parameter "voting". "voting" should be one of these options {selection, hybrid, weighting}

In [49]:
study.best_params

{'k': 19, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.23793040741239418}

In [15]:
trainf = pd.read_pickle(data_path/'train_final_OHE.pkl')
testf = pd.read_pickle(data_path/'test_final_OHE.pkl')
trainf_x = trainf.drop(['label'], axis = 1)
trainf_y = trainf['label']
testf_x = testf.drop(['label'], axis = 1)
testf_y = testf['label']

In [55]:
knu = des.KNORAU(classifiers, k = 19, with_IH = False, DFP = False, IH_rate=0.23793040741239418)

In [56]:
knu.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


KNORAU(IH_rate=0.23793040741239418, k=19,
       pool_classifiers=[RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1B51F665040),
                         GaussianNB(),
                         BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B56FFC8540),
                                           n_estimators=50, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1B56FFC8540)])

In [114]:
new_het_probs = probs(knu, testf_x)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


In [115]:
# performance of hetero ensemble using nn, rf and NB and POOL and KNORA-U
roc_auc_score(y_true = testf_y, y_score = new_het_probs)

0.8932043486924863

In [177]:
average_precision_score(y_true= test_Y, y_score= new_het_probs)

0.4117238064544785

In [83]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,20)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    voting = trial.suggest_categorical('voting', [True, False])
    kne = knora_e.KNORAE(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state = 42)
    kne.fit(X_train, y_train)
    probs = kne.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorae',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )  

[I 2023-10-30 14:47:45,585] A new study created in memory with name: heterogeneous_knorae


In [85]:
study.optimize(lambda trial: objective(trial, classifiers, train_x, train_y, validation_x, validation_y), n_trials= 30)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-pack

In [87]:
study.best_params

{'k': 19,
 'ih_ind': True,
 'dfp': False,
 'ih_rate': 0.49902251082966376,
 'voting': False}

In [121]:
kne = knora_e.KNORAE(classifiers, k = 19, with_IH = True, DFP = False, IH_rate = 0.49902251082966376, random_state=42)

In [122]:
kne.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\base.py:475: RuntimeWarning: IH_rate is bigger than the highest possible IH.
  warnings.warn("IH_rate is bigger than the highest possible IH.",
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


KNORAE(IH_rate=0.49902251082966376, k=19,
       pool_classifiers=[RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1B51F665040),
                         GaussianNB(),
                         BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B56FFC8540),
                                           n_estimators=50, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1B56FFC8540)],
       random_state=42, safe_k=19, with_IH=True)

In [123]:
het_kne_probs = probs(kne, testf_x)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


In [125]:
# performance of hetero ensemble using nn, rf and NB and POOL and KNORA-E
roc_auc_score(y_true = testf_y, y_score = het_kne_probs)

0.8566613990871392

In [178]:
average_precision_score(y_true= test_Y, y_score= het_kne_probs)

0.22564371449556567

In [19]:
# try upsizing the pooled classifier
rng = np.random.RandomState(42)
pool_classifiers = BaggingClassifier(DecisionTreeClassifier(random_state=rng),
                                     random_state=rng, n_estimators= 100, n_jobs= -1)

In [20]:
pool_classifiers.fit(train_x, train_y)

BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1F989404B40),
                  n_estimators=100, n_jobs=-1,
                  random_state=RandomState(MT19937) at 0x1F989404B40)

In [21]:
perf_report(probs(pool_classifiers, test_X), test_Y)

ROC_AUC is 0.8919952050504152
PR_AUC is 0.38672349804710643


In [22]:
model_filename = 'pool_100.pkl'

# Open the file in write-binary mode and save the model
with open(data_path/model_filename, 'wb') as file:
    pickle.dump(pool_classifiers, file)

In [132]:
# try adding a knn classifier
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7, weights = 'distance', n_jobs = -1)
neigh.fit(train_x, train_y)

KNeighborsClassifier(n_jobs=-1, n_neighbors=7, weights='distance')

In [134]:
L_pool_probs = probs(pool_classifiers, test_X)

In [135]:
# performance of 100 bagged trees
roc_auc_score(y_true = testf_y, y_score = L_pool_probs)

0.8919952050504152

In [179]:
average_precision_score(y_true= test_Y, y_score= L_pool_probs)

0.38672349804710643

In [140]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,20)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = des.KNORAU(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42)
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'homogeneous_knorau',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-10-30 16:08:38,427] A new study created in memory with name: homogeneous_knorau


In [141]:
study.optimize(lambda trial: objective(trial, pool_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

[I 2023-10-30 16:09:46,602] Trial 0 finished with value: 0.8749438454772198 and parameters: {'k': 18, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.41991598958386905}. Best is trial 0 with value: 0.8749438454772198.
[I 2023-10-30 16:10:50,206] Trial 1 finished with value: 0.8095132994834688 and parameters: {'k': 18, 'ih_ind': True, 'dfp': True, 'ih_rate': 0.4601683685757624}. Best is trial 0 with value: 0.8749438454772198.
[I 2023-10-30 16:11:48,681] Trial 2 finished with value: 0.8030062528819961 and parameters: {'k': 7, 'ih_ind': True, 'dfp': False, 'ih_rate': 0.38118374798116156}. Best is trial 0 with value: 0.8749438454772198.
[I 2023-10-30 16:12:21,013] Trial 3 finished with value: 0.8342882616058062 and parameters: {'k': 8, 'ih_ind': True, 'dfp': True, 'ih_rate': 0.11780882785553337}. Best is trial 0 with value: 0.8749438454772198.
[I 2023-10-30 16:12:51,310] Trial 4 finished with value: 0.8484972933455437 and parameters: {'k': 15, 'ih_ind': True, 'dfp': False, 'ih_rate': 0.0802079

In [143]:
study.best_params

{'k': 5, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.42013941411333383}

In [144]:
large_homo = des.KNORAU(pool_classifiers, k = 5, with_IH = False, DFP = False, random_state= 42)

In [145]:
large_homo.fit(trainf_x, trainf_y)

KNORAU(k=5,
       pool_classifiers=BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                          n_estimators=100, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x1B536812D40),
       random_state=42)

In [146]:
large_homo_p = probs(large_homo, testf_x)

In [147]:
# KNORAU 100 bagged trees
roc_auc_score(y_true = testf_y, y_score = large_homo_p)

0.892250721098477

In [183]:
average_precision_score(y_true = testf_y, y_score = large_homo_p)

0.38900615218763934

In [162]:
# try using 100 pooled DTs in knora-u
new_classifiers = [pool_classifiers, nn, rf, NB]


In [163]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,20)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    voting = trial.suggest_categorical('voting', [True, False])
    kne = knora_e.KNORAE(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state = 42)
    kne.fit(X_train, y_train)
    probs = kne.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorae',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )  

[I 2023-10-31 18:19:41,871] A new study created in memory with name: heterogeneous_knorae


In [164]:
study.optimize(lambda trial: objective(trial, new_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

In [165]:
study.best_params

{'k': 19,
 'ih_ind': True,
 'dfp': True,
 'ih_rate': 0.48442576184359243,
 'voting': True}

In [167]:
kne = knora_e.KNORAE(new_classifiers, k = 19, with_IH = True, DFP = True, IH_rate=0.48442576184359243, random_state = 42)

In [168]:
kne.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\base.py:475: RuntimeWarning: IH_rate is bigger than the highest possible IH.
  warnings.warn("IH_rate is bigger than the highest possible IH.",
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


KNORAE(DFP=True, IH_rate=0.48442576184359243, k=19,
       pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                           n_estimators=100, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1B536812D40),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1B51F665040),
                         RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         GaussianNB()],
       random_state=42, safe_k=19, with_IH=True)

In [169]:
large_kne_probs = probs(kne, testf_x)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [171]:
# KNORAE (100 bagged trees, nn, rf, NB)
roc_auc_score(y_true = testf_y, y_score = large_kne_probs)

0.8577112650407139

In [184]:
average_precision_score(y_true = testf_y, y_score = large_kne_probs)

0.2253622861483084

In [185]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,20)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = des.KNORAU(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42)
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorau',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-10-31 19:06:14,705] A new study created in memory with name: heterogeneous_knorau


In [186]:
study.optimize(lambda trial: objective(trial, new_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

In [189]:
study.best_params

{'k': 19, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.2975670784147758}

In [190]:
knu = des.KNORAU(new_classifiers, k = 19, with_IH = False, DFP = False, IH_rate=0.2975670784147758, random_state= 42)

In [191]:
knu.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


KNORAU(IH_rate=0.2975670784147758, k=19,
       pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                           n_estimators=100, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1B536812D40),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1B51F665040),
                         RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         GaussianNB()],
       random_state=42)

In [193]:
best_knu_probs = probs(knu, test_X) 

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [195]:
# KNORAU (100 bagged trees, nn, rf, NB)
roc_auc_score(y_true = testf_y, y_score = best_knu_probs)

0.8939318345854221

In [196]:
average_precision_score(y_true = testf_y, y_score = best_knu_probs)

0.4161210342204853

In [197]:
knn_probs = probs(neigh,testf_x)

In [205]:
perf_report(knn_probs, test_Y)

ROC_AUC is 0.8005969729318013
PR_AUC is 0.15994111483792048


In [207]:
most_classifiers = [neigh, nn, NB, rf, pool_classifiers]

In [210]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = des.KNORAU(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42)
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorau',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-10-31 23:10:26,464] A new study created in memory with name: heterogeneous_knorau


In [211]:
study.optimize(lambda trial: objective(trial, most_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packag

In [212]:
study.best_params

{'k': 19, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.3166105946899518}

In [222]:
bknu = des.KNORAU(new_classifiers, k = 19, with_IH = False, DFP = False, IH_rate=0.3166105946899518, random_state= 42)

In [223]:
bknu.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


KNORAU(IH_rate=0.3166105946899518, k=19,
       pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                           n_estimators=100, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1B536812D40),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1B51F665040),
                         RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         GaussianNB()],
       random_state=42)

In [224]:
bknu_probs = probs(bknu, test_X)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [225]:
perf_report(bknu_probs, test_Y)

ROC_AUC is 0.8939318345854221
PR_AUC is 0.4161210342204853


In [226]:
perf_report(probs(nn,test_X), test_Y)

ROC_AUC is 0.8882016624832005
PR_AUC is 0.3774998767083372


In [227]:
perf_report(probs(NB,test_X), test_Y)

ROC_AUC is 0.7796173405953397
PR_AUC is 0.11218142453121212


In [228]:
perf_report(probs(rf,test_X), test_Y)

ROC_AUC is 0.8878310370476331
PR_AUC is 0.3773455197352872


In [229]:
perf_report(probs(pool_classifiers,test_X), test_Y)

ROC_AUC is 0.8919952050504152
PR_AUC is 0.38672349804710643


In [230]:
import pandas as pd
import pickle
import optuna.integration.lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation

# Load the model from the 'model.pkl' file
loaded_model = pickle.load(open(data_path/'model.pkl', 'rb'))
test=pd.read_pickle(data_path/'test_final.pkl')
X_test=test.drop("label",axis=1)
y_test=test["label"]
categorical_features = ["sequence", "m1_seq", "p1_seq"]
for c in categorical_features:
    X_test[c]=X_test[c].astype("category")
predictions = loaded_model.predict(X_test,raw_scores=True)

[LightGBM] [Warning] Unknown parameter: raw_scores


In [239]:
loaded_model.params

{'objective': 'binary',
 'metric': 'binary_logloss',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 6.929425894844475,
 'lambda_l2': 1.1426652031102964e-08,
 'num_leaves': 247,
 'feature_fraction': 0.7799999999999999,
 'bagging_fraction': 0.9901149131368316,
 'bagging_freq': 1,
 'min_child_samples': 20,
 'categorical_column': [0, 1, 2],
 'num_iterations': 1000}

In [231]:
perf_report(predictions, test_Y)

ROC_AUC is 0.9116249182061309
PR_AUC is 0.4259408207595274


In [237]:
from sklearn.ensemble import VotingClassifier
voting_classifiers = [("perceptron", nn),
                      ("bayes", NB),
                   ("tree", pool_classifiers),
                      ("knn", neigh),
                    #  ("random_forest"), rf
                    ]
model_voting = VotingClassifier(estimators=voting_classifiers, voting = 'soft').fit(
    trainf_x, trainf_y)


In [238]:
perf_report(probs(model_voting,test_X), test_Y)

ROC_AUC is 0.881317930846924
PR_AUC is 0.3817898206586108


In [244]:
stacked_dt = static.StackedClassifier(new_classifiers,
                               random_state=42,
                               meta_classifier=DecisionTreeClassifier())

In [245]:
stacked_dt.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


StackedClassifier(meta_classifier=DecisionTreeClassifier(),
                  pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                                      n_estimators=100,
                                                      n_jobs=-1,
                                                      random_state=RandomState(MT19937) at 0x1B536812D40),
                                    MLPClassifier(max_iter=300,
                                                  random_state=RandomState(MT19937) at 0x1B51F665040),
                                    RandomForestClassifier(max_depth=11,
                                                           n_estimators=368,
                                                           n_jobs=-1,
                                                           random_state=42),
                                    GaussianNB()],
                  random_state=42)

In [246]:
perf_report(probs(stacked_dt, test_X),test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ROC_AUC is 0.7518043964369635
PR_AUC is 0.20847399064942276


c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [247]:
stacked_lr = static.StackedClassifier(new_classifiers,
                               random_state=42)

In [248]:
stacked_lr.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


StackedClassifier(pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                                      n_estimators=100,
                                                      n_jobs=-1,
                                                      random_state=RandomState(MT19937) at 0x1B536812D40),
                                    MLPClassifier(max_iter=300,
                                                  random_state=RandomState(MT19937) at 0x1B51F665040),
                                    RandomForestClassifier(max_depth=11,
                                                           n_estimators=368,
                                                           n_jobs=-1,
                                                           random_state=42),
                                    GaussianNB()],
                  random_state=42)

In [249]:
perf_report(probs(stacked_lr, test_X),test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8810642591894527
PR_AUC is 0.398491036040853


In [250]:
stacked_lr = static.StackedClassifier(most_classifiers,
                               random_state=42)

In [251]:
stacked_lr.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


StackedClassifier(pool_classifiers=[KNeighborsClassifier(n_jobs=-1,
                                                         n_neighbors=7,
                                                         weights='distance'),
                                    MLPClassifier(max_iter=300,
                                                  random_state=RandomState(MT19937) at 0x1B51F665040),
                                    GaussianNB(),
                                    RandomForestClassifier(max_depth=11,
                                                           n_estimators=368,
                                                           n_jobs=-1,
                                                           random_state=42),
                                    BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                                      n_estimators=100,
                                                      n_jobs=-1,
                                                      random_state=RandomState(MT19937) at 0x1B536812D40)],
                  random_state=42)

In [252]:
perf_report(probs(stacked_lr, test_X),test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8854886998265574
PR_AUC is 0.3807323312788329


In [264]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = desp.DESP(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_desp',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-11-01 09:41:30,119] A new study created in memory with name: heterogeneous_desp


In [255]:
study.optimize(lambda trial: objective(trial, new_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

In [257]:
study.best_params

{'k': 10, 'ih_ind': False, 'dfp': True, 'ih_rate': 0.20646492058689186}

In [269]:
desp_model = desp.DESP(new_classifiers, k = 10, with_IH = False, DFP = True, IH_rate=0.20646492058689186, random_state= 42, voting = 'soft')

AttributeError: 'DESP' object has no attribute 'DESP'

In [260]:
desp.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


DESP(DFP=True, IH_rate=0.20646492058689186, k=10,
     pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                         n_estimators=100, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x1B536812D40),
                       MLPClassifier(max_iter=300,
                                     random_state=RandomState(MT19937) at 0x1B51F665040),
                       RandomForestClassifier(max_depth=11, n_estimators=368,
                                              n_jobs=-1, random_state=42),
                       GaussianNB()],
     random_state=42, voting='soft')

In [261]:
perf_report(probs(desp, test_X),test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8990381561312746
PR_AUC is 0.42561512367948906


In [271]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = desp.DESP(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_desp',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-11-01 09:43:01,132] A new study created in memory with name: heterogeneous_desp


In [272]:
study.optimize(lambda trial: objective(trial, most_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packag

In [273]:
study.best_params

{'k': 20, 'ih_ind': False, 'dfp': True, 'ih_rate': 0.061371970722223765}

In [277]:
desp_B = desp.DESP(most_classifiers, k = 20, with_IH = False, DFP = True, IH_rate=0.061371970722223765, random_state= 42, voting = 'soft')

In [278]:
desp_B.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


DESP(DFP=True, IH_rate=0.061371970722223765, k=20,
     pool_classifiers=[KNeighborsClassifier(n_jobs=-1, n_neighbors=7,
                                            weights='distance'),
                       MLPClassifier(max_iter=300,
                                     random_state=RandomState(MT19937) at 0x1B51F665040),
                       GaussianNB(),
                       RandomForestClassifier(max_depth=11, n_estimators=368,
                                              n_jobs=-1, random_state=42),
                       BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                         n_estimators=100, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x1B536812D40)],
     random_state=42, voting='soft')

In [279]:
perf_report(probs(desp_B, test_X),test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(


ROC_AUC is 0.895051729137535
PR_AUC is 0.3823656357587157


In [298]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = desp.DESP(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return average_precision_score(y_true=y_valid,y_score=true_probs), roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_desp', directions = ["maximize","maximize"] )    

[I 2023-11-01 12:25:01,700] A new study created in memory with name: heterogeneous_desp


In [299]:
study.optimize(lambda trial: objective(trial, new_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 60)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

In [300]:
study.best_trials

[FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.45191029760574314, 0.9123476659222454], datetime_start=datetime.datetime(2023, 11, 1, 12, 25, 24, 609745), datetime_complete=datetime.datetime(2023, 11, 1, 12, 25, 37, 301827), params={'k': 17, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.024209491754130785}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'k': IntDistribution(high=25, log=False, low=5, step=1), 'ih_ind': CategoricalDistribution(choices=(True, False)), 'dfp': CategoricalDistribution(choices=(True, False)), 'ih_rate': FloatDistribution(high=0.5, log=False, low=0.0, step=None)}, trial_id=1, value=None),
 FrozenTrial(number=24, state=TrialState.COMPLETE, values=[0.451030886349055, 0.9165993608911357], datetime_start=datetime.datetime(2023, 11, 1, 12, 30, 32, 320569), datetime_complete=datetime.datetime(2023, 11, 1, 12, 30, 44, 200979), params={'k': 14, 'ih_ind': False, 'dfp': True, 'ih_rate': 0.02143242982711463}, user_a

In [313]:
opt = desp.DESP(new_classifiers, k = 14, with_IH = False, DFP = True, IH_rate=0.2650472926942986, random_state= 42, voting = 'soft')

In [314]:
opt.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


DESP(DFP=True, IH_rate=0.2650472926942986, k=14,
     pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                         n_estimators=100, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x1B536812D40),
                       MLPClassifier(max_iter=300,
                                     random_state=RandomState(MT19937) at 0x1B51F665040),
                       RandomForestClassifier(max_depth=11, n_estimators=368,
                                              n_jobs=-1, random_state=42),
                       GaussianNB()],
     random_state=42, voting='soft')

In [315]:
perf_report(probs(opt, test_X), test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8952215878036313
PR_AUC is 0.419030986253681


In [319]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    Kp = trial.suggest_int('Kp',3,7)
    selection_threshold = trial.suggest_float('selection_threshold',0.5,1.0)
    knu = meta.METADES(pool, k = k, with_IH = ih_ind, DFP = True, Kp = Kp, selection_threshold = selection_threshold, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_METADES', direction = 'maximize')    

[I 2023-11-01 13:09:28,922] A new study created in memory with name: heterogeneous_METADES


In [320]:
study.optimize(lambda trial: objective(trial, new_classifiers, train_x, train_y, validation_x, validation_y), n_trials= 70)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

In [321]:
study.best_params

{'k': 24,
 'ih_ind': False,
 'ih_rate': 0.4940692081713565,
 'Kp': 5,
 'selection_threshold': 0.6544931130898003}

In [322]:
metaD = meta.METADES(new_classifiers, k = 24, with_IH = False, DFP = True, Kp = 5, selection_threshold = 0.6544931130898003, IH_rate=0.2, random_state= 42, voting = 'soft')

In [323]:
metaD.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\

METADES(DFP=True, IH_rate=0.2, k=24,
        pool_classifiers=[BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1B536812D40),
                                            n_estimators=100, n_jobs=-1,
                                            random_state=RandomState(MT19937) at 0x1B536812D40),
                          MLPClassifier(max_iter=300,
                                        random_state=RandomState(MT19937) at 0x1B51F665040),
                          RandomForestClassifier(max_depth=11, n_estimators=368,
                                                 n_jobs=-1, random_state=42),
                          GaussianNB()],
        random_state=42, selection_threshold=0.6544931130898003, voting='soft')

In [324]:
perf_report(probs(metaD, test_X), test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8880722833369133
PR_AUC is 0.3801563683493936


In [3]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    n_estimators = trial.suggest_int('n_estimators',50, 350)
    learning_rate = trial.suggest_float('lr', 0.01, 10) 
    adaB = AdaBoostClassifier(n_estimators= n_estimators, learning_rate= learning_rate, random_state= 42)
    adaB.fit(X_train, y_train)
    probs = adaB.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'adaB', direction = 'maximize')    

[I 2023-11-01 16:25:02,610] A new study created in memory with name: adaB


In [11]:
study.optimize(lambda trial: objective(trial, train_x, train_y, validation_x, validation_y), n_trials= 40, n_jobs= -1)

[I 2023-11-01 16:33:12,315] Trial 2 finished with value: 0.6678811952420748 and parameters: {'n_estimators': 70, 'lr': 6.928955093144314}. Best is trial 2 with value: 0.6678811952420748.
[I 2023-11-01 16:36:05,356] Trial 4 finished with value: 0.6678811952420748 and parameters: {'n_estimators': 128, 'lr': 27.72602963144483}. Best is trial 2 with value: 0.6678811952420748.
[I 2023-11-01 16:36:09,993] Trial 9 finished with value: 0.7766286749082605 and parameters: {'n_estimators': 134, 'lr': 0.011315048325636646}. Best is trial 9 with value: 0.7766286749082605.
[I 2023-11-01 16:38:22,429] Trial 8 finished with value: 0.7911327520043192 and parameters: {'n_estimators': 179, 'lr': 0.01285761480905331}. Best is trial 8 with value: 0.7911327520043192.
[I 2023-11-01 16:38:59,739] Trial 6 finished with value: 0.8835080721722856 and parameters: {'n_estimators': 197, 'lr': 0.7582447017231118}. Best is trial 6 with value: 0.8835080721722856.
[I 2023-11-01 16:41:24,417] Trial 5 finished with value

In [12]:
study.best_params

{'n_estimators': 205, 'lr': 0.730555370099575}

In [13]:
adaB = AdaBoostClassifier(n_estimators = 205, learning_rate = 0.730555370099575)

In [16]:
adaB.fit(trainf_x, trainf_y)

AdaBoostClassifier(learning_rate=0.730555370099575, n_estimators=205)

In [17]:
perf_report(probs(adaB, test_X), test_Y)

ROC_AUC is 0.8944882386900804
PR_AUC is 0.399560486943237


In [18]:
model_filename = 'adaboost.pkl'

# Open the file in write-binary mode and save the model
with open(data_path/model_filename, 'wb') as file:
    pickle.dump(adaB, file)

In [26]:
class_v0 = [adaB, pool_classifiers, rf, nn, NB]

In [42]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    desp_model = desp.DESP(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    desp_model.fit(X_train, y_train)
    probs = desp_model.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_desp',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-11-01 23:55:35,456] A new study created in memory with name: heterogeneous_desp


In [43]:
study.optimize(lambda trial: objective(trial, class_v0, train_x, train_y, validation_x, validation_y), n_trials= 60, n_jobs= -1)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\base.py:475: RuntimeWarning: IH_rate is bigger than the highest possible IH.
  warnings.warn("IH_rate is bigger than the highest possible IH.",
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\

In [44]:
study.best_params

{'k': 6, 'ih_ind': False, 'dfp': True, 'ih_rate': 0.4565716319490343}

In [45]:
desp_model = desp.DESP(class_v0, k = 6, with_IH = False, DFP = True, IH_rate=0.4, random_state= 42, voting = 'soft')

In [46]:
desp_model.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


DESP(DFP=True, IH_rate=0.4, k=6,
     pool_classifiers=[AdaBoostClassifier(learning_rate=0.730555370099575,
                                          n_estimators=205),
                       BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1F989404B40),
                                         n_estimators=100, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x1F989404B40),
                       RandomForestClassifier(max_depth=11, n_estimators=368,
                                              n_jobs=-1, random_state=42),
                       MLPClassifier(max_iter=300,
                                     random_state=RandomState(MT19937) at 0x1F9EC979840),
                       GaussianNB()],
     random_state=42, voting='soft')

In [47]:
perf_report(probs(desp_model, test_X), test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8961270670959012
PR_AUC is 0.407286596404152


In [48]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    dfp = trial.suggest_categorical('dfp', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    knu = des.KNORAU(pool, k = k, with_IH = ih_ind, DFP = dfp, IH_rate=ih_rate, random_state= 42)
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_knorau',pruner = optuna.pruners.HyperbandPruner(
        min_resource=1, reduction_factor=3
    ), direction = "maximize", )    

[I 2023-11-02 00:13:51,606] A new study created in memory with name: heterogeneous_knorau


In [49]:
study.optimize(lambda trial: objective(trial, class_v0, train_x, train_y, validation_x, validation_y), n_trials= 60, n_jobs= -1)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\sit

In [50]:
study.best_params

{'k': 18, 'ih_ind': False, 'dfp': False, 'ih_rate': 0.2107380391285169}

In [51]:
knu_v0 = des.KNORAU(class_v0, k = 18, with_IH = False, DFP = False, IH_rate=0.03, random_state= 42)

In [52]:
knu_v0.fit(trainf_x, trainf_y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


KNORAU(IH_rate=0.03, k=18,
       pool_classifiers=[AdaBoostClassifier(learning_rate=0.730555370099575,
                                            n_estimators=205),
                         BaggingClassifier(estimator=DecisionTreeClassifier(random_state=RandomState(MT19937) at 0x1F989404B40),
                                           n_estimators=100, n_jobs=-1,
                                           random_state=RandomState(MT19937) at 0x1F989404B40),
                         RandomForestClassifier(max_depth=11, n_estimators=368,
                                                n_jobs=-1, random_state=42),
                         MLPClassifier(max_iter=300,
                                       random_state=RandomState(MT19937) at 0x1F9EC979840),
                         GaussianNB()],
       random_state=42)

In [53]:
perf_report(probs(knu_v0,test_X), test_Y)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ROC_AUC is 0.8987165332415098
PR_AUC is 0.3985937736713405


In [54]:
def objective(trial, pool, X_train, y_train, X_valid, y_valid):
    k = trial.suggest_int('k', 5,25)
    ih_ind = trial.suggest_categorical('ih_ind', [True, False])
    ih_rate = trial.suggest_float('ih_rate', 0.0,0.5)
    # Kp = trial.suggest_int('Kp',3,7)
    selection_threshold = trial.suggest_float('selection_threshold',0.5,1.0)
    knu = meta.METADES(pool, k = k, with_IH = ih_ind, DFP = True, Kp = 5, selection_threshold = selection_threshold, IH_rate=ih_rate, random_state= 42, voting = 'soft')
    knu.fit(X_train, y_train)
    probs = knu.predict_proba(X_valid)
    true_probs = [entry[1] for entry in probs]
    return roc_auc_score(y_true=y_valid,y_score=true_probs)

study = optuna.create_study(study_name = 'heterogeneous_METADES', direction = 'maximize')    

[I 2023-11-02 01:07:36,305] A new study created in memory with name: heterogeneous_METADES


In [55]:
study.optimize(lambda trial: objective(trial, class_v0, train_x, train_y, validation_x, validation_y), n_trials= 60, n_jobs= -1)

c:\Users\65829\anaconda3\envs\wp\lib\site-packages\deslib\base.py:475: RuntimeWarning: IH_rate is bigger than the highest possible IH.
  warnings.warn("IH_rate is bigger than the highest possible IH.",
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\wp\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\65829\anaconda3\envs\